In [1]:
import random
from collections import Counter
import stox
# from ib_insync import *
from stox.imports import *
from stox.ibkr import IB_Extractor, describe_contract, download_reports, IBKR_Loader, IBKR_Stats, IBKR_Derived
from tqdm.notebook import tqdm
from omnibelt import tqdmd_notebook as tqdmd
from stox import misc, yahoo, ibkr
from collections import namedtuple
from omniply import tool, ToolKit, Context

In [2]:
symbol_table = stox.load_symbol_table()
len(symbol_table)

272

In [3]:
rows = [(k, v) for k, v in symbol_table.items() if v['currency'] == 'EUR']
len(rows)

239

In [4]:
date = '230829'
# date = 'last'

In [68]:
yfsym, ibrow = random.choice(rows)
gg = Context(IBKR_Derived(), IBKR_Stats(), IBKR_Loader(date=date, root=misc.ibkr_root()))
gg.update(ibrow)
gg['ibsym'] = ibrow['symbol']
gg['ibid'] = ibrow['conId']
gg['yfsym'] = yfsym
print(yfsym)

QUEST.AT


In [69]:
gg

Context(conId, currency, exchange, primaryExchange, symbol, ibsym, ibid, yfsym, {sector}, {company_name}, {exchange_name}, {country}, {isin}, {price}, {market_cap}, {high_52w}, {low_52w}, {employees}, {employees_date}, {business_summary}, {brief}, {brief_date}, {country_code}, {country_name}, {city}, {website}, {industry_trbc}, {industry}, {industry_trbc_code}, {industry_naics}, {industry_naics_code}, {industry_sic}, {industry_sic_code}, {ckpt_date}, {ckpt_path}, {snapshot}, {ownership}, {finances}, {statements}, {recommendations})

In [70]:
gg['company_name']

'Quest Holdings SA'

In [73]:
gg['price'], gg['high_52w'], gg['low_52w'], gg['market_cap']

AssertionError: Expected CLPRICE in {'@type': 'MARKETCAP', '@unit': 'M', '@currCode': 'EUR', '#text': '654.06'}

In [77]:
gg['sector'], gg['industry']

('Technology', 'IT Services & Consulting (NEC)')

In [78]:
gg['city'], gg['country_name'], gg['employees']

('Athina', 'Greece', 2621)

In [76]:
gg['industry_naics']

['Custom Computer Programming Services',
 'Web Search Portals and All Other Information Services',
 'Computing Infrastructure Providers, Data Processing, Web Hosting, and Related Services',
 'Computer Systems Design Services',
 'Postal Service',
 'All Other Miscellaneous Retailers',
 'Offices of Other Holding Companies']

In [10]:
list(gg.vendors('exchange'))

[_ToolSkill(IBKR_Stats.get_exchange_from_snapshot: exchange),
 _ToolSkill(IBKR_Stats.get_exchange_from_rec: exchange)]